## Visualización de la Distribución de Mortalidad Intrahospitalaria

Visualización de los datos reducidos a 2D para observar la distribución de las clases y la frontera de decisión.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Cargar el dataset
df = pd.read_csv('cleaned_dataset_20251104_200505.csv')

# Separar features y target
target = 'mortality_inhospital'
X = df.drop(columns=[target])
y = df[target]

# Manejar valores faltantes para el análisis
X_filled = X.fillna(X.median())

# Estandarizar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_filled)

print(f"Shape del dataset: {df.shape}")
print(f"Distribución de clases:")
print(y.value_counts())
print(f"\nPorcentaje de mortalidad: {(y.sum()/len(y))*100:.2f}%")

Shape del dataset: (3093, 132)
Distribución de clases:
mortality_inhospital
0    2819
1     274
Name: count, dtype: int64

Porcentaje de mortalidad: 8.86%


In [7]:
# Reducción de dimensionalidad con PCA a 2 componentes
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print(f"Varianza explicada por cada componente: {pca.explained_variance_ratio_}")
print(f"Varianza total explicada: {sum(pca.explained_variance_ratio_)*100:.2f}%")

# Crear DataFrame con los resultados de PCA
df_pca = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'Mortalidad': y.map({0: 'Sobrevivió', 1: 'Falleció'})
})

# Visualización interactiva con Plotly
fig = px.scatter(
    df_pca, 
    x='PC1', 
    y='PC2', 
    color='Mortalidad',
    color_discrete_map={'Sobrevivió': '#2ecc71', 'Falleció': '#e74c3c'},
    title='Distribución de Mortalidad Intrahospitalaria (PCA 2D)',
    labels={'PC1': f'Componente Principal 1 ({pca.explained_variance_ratio_[0]*100:.1f}%)',
            'PC2': f'Componente Principal 2 ({pca.explained_variance_ratio_[1]*100:.1f}%)'},
    opacity=0.7
)

fig.update_layout(
    template='plotly_white',
    width=900,
    height=600,
    legend=dict(
        title='Estado',
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

fig.show()

Varianza explicada por cada componente: [0.06238425 0.05332045]
Varianza total explicada: 11.57%


In [8]:
# t-SNE para mejor separación visual de clusters
print("Calculando t-SNE (puede tomar unos segundos)...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_scaled)

# Crear DataFrame con los resultados de t-SNE
df_tsne = pd.DataFrame({
    'TSNE1': X_tsne[:, 0],
    'TSNE2': X_tsne[:, 1],
    'Mortalidad': y.map({0: 'Sobrevivió', 1: 'Falleció'})
})

# Visualización interactiva con Plotly
fig_tsne = px.scatter(
    df_tsne, 
    x='TSNE1', 
    y='TSNE2', 
    color='Mortalidad',
    color_discrete_map={'Sobrevivió': '#2ecc71', 'Falleció': '#e74c3c'},
    title='Distribución de Mortalidad Intrahospitalaria (t-SNE 2D)',
    labels={'TSNE1': 't-SNE Dimensión 1',
            'TSNE2': 't-SNE Dimensión 2'},
    opacity=0.7
)

fig_tsne.update_layout(
    template='plotly_white',
    width=900,
    height=600,
    legend=dict(
        title='Estado',
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

fig_tsne.show()

Calculando t-SNE (puede tomar unos segundos)...


In [9]:
# Visualización con frontera de decisión en t-SNE 2D
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

# Entrenar un clasificador SVM en el espacio t-SNE
clf_tsne = SVC(kernel='rbf', C=1.0, gamma='scale', probability=True)
clf_tsne.fit(X_tsne, y)

# Crear meshgrid para la frontera de decisión en t-SNE
x_min_tsne, x_max_tsne = X_tsne[:, 0].min() - 2, X_tsne[:, 0].max() + 2
y_min_tsne, y_max_tsne = X_tsne[:, 1].min() - 2, X_tsne[:, 1].max() + 2
xx_tsne, yy_tsne = np.meshgrid(np.linspace(x_min_tsne, x_max_tsne, 200),
                               np.linspace(y_min_tsne, y_max_tsne, 200))

# Predecir probabilidades en el meshgrid
Z_tsne = clf_tsne.predict_proba(np.c_[xx_tsne.ravel(), yy_tsne.ravel()])[:, 1]
Z_tsne = Z_tsne.reshape(xx_tsne.shape)

# Crear figura con frontera de decisión en t-SNE
fig_boundary_tsne = go.Figure()

# Agregar contorno de probabilidad (frontera de decisión)
fig_boundary_tsne.add_trace(go.Contour(
    x=np.linspace(x_min_tsne, x_max_tsne, 200),
    y=np.linspace(y_min_tsne, y_max_tsne, 200),
    z=Z_tsne,
    colorscale=[[0, 'rgba(46, 204, 113, 0.3)'], [0.5, 'rgba(255, 255, 255, 0.1)'], [1, 'rgba(231, 76, 60, 0.3)']],
    showscale=True,
    colorbar=dict(title='Probabilidad<br>de Mortalidad'),
    contours=dict(
        showlines=True,
        start=0,
        end=1,
        size=0.1
    ),
    line=dict(width=1),
    name='Probabilidad'
))

# Agregar línea de frontera de decisión (probabilidad = 0.5)
fig_boundary_tsne.add_trace(go.Contour(
    x=np.linspace(x_min_tsne, x_max_tsne, 200),
    y=np.linspace(y_min_tsne, y_max_tsne, 200),
    z=Z_tsne,
    showscale=False,
    contours=dict(
        showlines=True,
        start=0.5,
        end=0.5,
        size=0.1,
        coloring='none'
    ),
    line=dict(width=3, color='black', dash='dash'),
    name='Frontera (p=0.5)'
))

# Agregar puntos de sobrevivientes
sobrevivio_mask = y == 0
fig_boundary_tsne.add_trace(go.Scatter(
    x=X_tsne[sobrevivio_mask, 0],
    y=X_tsne[sobrevivio_mask, 1],
    mode='markers',
    marker=dict(
        color='#2ecc71',
        size=6,
        opacity=0.7,
        line=dict(width=0.5, color='white')
    ),
    name='Sobrevivió'
))

# Agregar puntos de fallecidos
fallecio_mask = y == 1
fig_boundary_tsne.add_trace(go.Scatter(
    x=X_tsne[fallecio_mask, 0],
    y=X_tsne[fallecio_mask, 1],
    mode='markers',
    marker=dict(
        color='#e74c3c',
        size=8,
        opacity=0.8,
        symbol='x',
        line=dict(width=1, color='darkred')
    ),
    name='Falleció'
))

fig_boundary_tsne.update_layout(
    title='Distribución con Frontera de Decisión (SVM-RBF en t-SNE 2D)',
    xaxis_title='t-SNE Dimensión 1',
    yaxis_title='t-SNE Dimensión 2',
    template='plotly_white',
    width=1000,
    height=700,
    legend=dict(
        title='Clase',
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

fig_boundary_tsne.show()

print(f"\nPrecisión del clasificador SVM en espacio t-SNE: {clf_tsne.score(X_tsne, y)*100:.2f}%")


Precisión del clasificador SVM en espacio t-SNE: 91.14%


In [10]:
# Visualización con frontera de decisión usando un clasificador simple
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')

# Entrenar un clasificador en el espacio PCA para visualizar la frontera
clf = SVC(kernel='rbf', C=1.0, gamma='scale', probability=True)
clf.fit(X_pca, y)

# Crear meshgrid para la frontera de decisión
x_min, x_max = X_pca[:, 0].min() - 1, X_pca[:, 0].max() + 1
y_min, y_max = X_pca[:, 1].min() - 1, X_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                      np.linspace(y_min, y_max, 200))

# Predecir probabilidades en el meshgrid
Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
Z = Z.reshape(xx.shape)

# Crear figura con frontera de decisión
fig_boundary = go.Figure()

# Agregar contorno de probabilidad (frontera de decisión)
fig_boundary.add_trace(go.Contour(
    x=np.linspace(x_min, x_max, 200),
    y=np.linspace(y_min, y_max, 200),
    z=Z,
    colorscale=[[0, 'rgba(46, 204, 113, 0.3)'], [0.5, 'rgba(255, 255, 255, 0.1)'], [1, 'rgba(231, 76, 60, 0.3)']],
    showscale=True,
    colorbar=dict(title='Probabilidad<br>de Mortalidad'),
    contours=dict(
        showlines=True,
        start=0,
        end=1,
        size=0.1
    ),
    line=dict(width=1),
    name='Probabilidad'
))

# Agregar línea de frontera de decisión (probabilidad = 0.5)
fig_boundary.add_trace(go.Contour(
    x=np.linspace(x_min, x_max, 200),
    y=np.linspace(y_min, y_max, 200),
    z=Z,
    showscale=False,
    contours=dict(
        showlines=True,
        start=0.5,
        end=0.5,
        size=0.1,
        coloring='none'
    ),
    line=dict(width=3, color='black', dash='dash'),
    name='Frontera (p=0.5)'
))

# Agregar puntos de sobrevivientes
sobrevivio_mask = y == 0
fig_boundary.add_trace(go.Scatter(
    x=X_pca[sobrevivio_mask, 0],
    y=X_pca[sobrevivio_mask, 1],
    mode='markers',
    marker=dict(
        color='#2ecc71',
        size=6,
        opacity=0.7,
        line=dict(width=0.5, color='white')
    ),
    name='Sobrevivió'
))

# Agregar puntos de fallecidos
fallecio_mask = y == 1
fig_boundary.add_trace(go.Scatter(
    x=X_pca[fallecio_mask, 0],
    y=X_pca[fallecio_mask, 1],
    mode='markers',
    marker=dict(
        color='#e74c3c',
        size=8,
        opacity=0.8,
        symbol='x',
        line=dict(width=1, color='darkred')
    ),
    name='Falleció'
))

fig_boundary.update_layout(
    title='Distribución con Frontera de Decisión (SVM-RBF en PCA 2D)',
    xaxis_title=f'Componente Principal 1 ({pca.explained_variance_ratio_[0]*100:.1f}%)',
    yaxis_title=f'Componente Principal 2 ({pca.explained_variance_ratio_[1]*100:.1f}%)',
    template='plotly_white',
    width=1000,
    height=700,
    legend=dict(
        title='Clase',
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

fig_boundary.show()

print(f"\nPrecisión del clasificador SVM en espacio PCA: {clf.score(X_pca, y)*100:.2f}%")


Precisión del clasificador SVM en espacio PCA: 91.14%


In [11]:
# Comparación lado a lado: PCA vs t-SNE
fig_comparison = make_subplots(
    rows=1, cols=2,
    subplot_titles=('PCA - 2 Componentes', 't-SNE - 2 Dimensiones'),
    horizontal_spacing=0.1
)

# Colores por clase
colors = ['#2ecc71' if m == 0 else '#e74c3c' for m in y]

# PCA subplot
fig_comparison.add_trace(
    go.Scatter(
        x=X_pca[:, 0], 
        y=X_pca[:, 1],
        mode='markers',
        marker=dict(color=colors, size=5, opacity=0.6),
        showlegend=False
    ),
    row=1, col=1
)

# t-SNE subplot
fig_comparison.add_trace(
    go.Scatter(
        x=X_tsne[:, 0], 
        y=X_tsne[:, 1],
        mode='markers',
        marker=dict(color=colors, size=5, opacity=0.6),
        showlegend=False
    ),
    row=1, col=2
)

# Agregar leyenda manual
fig_comparison.add_trace(
    go.Scatter(x=[None], y=[None], mode='markers',
               marker=dict(size=10, color='#2ecc71'),
               legendgroup='sobrevivio', showlegend=True, name='Sobrevivió')
)
fig_comparison.add_trace(
    go.Scatter(x=[None], y=[None], mode='markers',
               marker=dict(size=10, color='#e74c3c'),
               legendgroup='fallecio', showlegend=True, name='Falleció')
)

fig_comparison.update_layout(
    title_text='Comparación de Técnicas de Reducción de Dimensionalidad',
    template='plotly_white',
    width=1200,
    height=550,
    legend=dict(
        title='Estado',
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

fig_comparison.update_xaxes(title_text='PC1', row=1, col=1)
fig_comparison.update_yaxes(title_text='PC2', row=1, col=1)
fig_comparison.update_xaxes(title_text='t-SNE 1', row=1, col=2)
fig_comparison.update_yaxes(title_text='t-SNE 2', row=1, col=2)

fig_comparison.show()

In [12]:
# Análisis de densidad por clase
fig_density = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Densidad PC1 por Clase', 
        'Densidad PC2 por Clase',
        'Densidad t-SNE1 por Clase',
        'Densidad t-SNE2 por Clase'
    ),
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# Datos por clase
sobrevivio = y == 0
fallecio = y == 1

# PC1
fig_density.add_trace(go.Histogram(x=X_pca[sobrevivio, 0], name='Sobrevivió', 
                                    marker_color='#2ecc71', opacity=0.7, nbinsx=50), row=1, col=1)
fig_density.add_trace(go.Histogram(x=X_pca[fallecio, 0], name='Falleció', 
                                    marker_color='#e74c3c', opacity=0.7, nbinsx=50), row=1, col=1)

# PC2
fig_density.add_trace(go.Histogram(x=X_pca[sobrevivio, 1], name='Sobrevivió', 
                                    marker_color='#2ecc71', opacity=0.7, showlegend=False, nbinsx=50), row=1, col=2)
fig_density.add_trace(go.Histogram(x=X_pca[fallecio, 1], name='Falleció', 
                                    marker_color='#e74c3c', opacity=0.7, showlegend=False, nbinsx=50), row=1, col=2)

# t-SNE1
fig_density.add_trace(go.Histogram(x=X_tsne[sobrevivio, 0], name='Sobrevivió', 
                                    marker_color='#2ecc71', opacity=0.7, showlegend=False, nbinsx=50), row=2, col=1)
fig_density.add_trace(go.Histogram(x=X_tsne[fallecio, 0], name='Falleció', 
                                    marker_color='#e74c3c', opacity=0.7, showlegend=False, nbinsx=50), row=2, col=1)

# t-SNE2
fig_density.add_trace(go.Histogram(x=X_tsne[sobrevivio, 1], name='Sobrevivió', 
                                    marker_color='#2ecc71', opacity=0.7, showlegend=False, nbinsx=50), row=2, col=2)
fig_density.add_trace(go.Histogram(x=X_tsne[fallecio, 1], name='Falleció', 
                                    marker_color='#e74c3c', opacity=0.7, showlegend=False, nbinsx=50), row=2, col=2)

fig_density.update_layout(
    title_text='Distribución de las Dimensiones Reducidas por Clase',
    template='plotly_white',
    width=1100,
    height=700,
    barmode='overlay',
    legend=dict(
        title='Estado',
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

fig_density.show()

# Resumen estadístico
print("\n=== Resumen de separación de clases ===")
print(f"\nTotal de muestras: {len(y)}")
print(f"Sobrevivientes: {(y==0).sum()} ({(y==0).sum()/len(y)*100:.1f}%)")
print(f"Fallecidos: {(y==1).sum()} ({(y==1).sum()/len(y)*100:.1f}%)")


=== Resumen de separación de clases ===

Total de muestras: 3093
Sobrevivientes: 2819 (91.1%)
Fallecidos: 274 (8.9%)


## Visualización después de SMOTEENN

Aplicamos SMOTEENN (combinación de SMOTE + Edited Nearest Neighbors) para balancear las clases y luego visualizamos con PCA y t-SNE.

In [13]:
# import numpy as np
from imblearn.combine import SMOTEENN

# Aplicar SMOTEENN a los datos escalados
print("Aplicando SMOTEENN...")
print(f"Distribución original: {dict(zip(*np.unique(y, return_counts=True)))}")

smoteenn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smoteenn.fit_resample(X_scaled, y)

print(f"Distribución después de SMOTEENN: {dict(zip(*np.unique(y_resampled, return_counts=True)))}")
print(f"Shape original: {X_scaled.shape}")
print(f"Shape después de SMOTEENN: {X_resampled.shape}")

Aplicando SMOTEENN...
Distribución original: {np.int64(0): np.int64(2819), np.int64(1): np.int64(274)}
Distribución original: {np.int64(0): np.int64(2819), np.int64(1): np.int64(274)}
Distribución después de SMOTEENN: {np.int64(0): np.int64(642), np.int64(1): np.int64(2788)}
Shape original: (3093, 131)
Shape después de SMOTEENN: (3430, 131)
Distribución después de SMOTEENN: {np.int64(0): np.int64(642), np.int64(1): np.int64(2788)}
Shape original: (3093, 131)
Shape después de SMOTEENN: (3430, 131)


In [14]:
# PCA sobre datos con SMOTEENN
print("Aplicando PCA a datos balanceados con SMOTEENN...")
pca_smoteenn = PCA(n_components=2)
X_pca_smoteenn = pca_smoteenn.fit_transform(X_resampled)

print(f"Varianza explicada: {pca_smoteenn.explained_variance_ratio_}")
print(f"Varianza total explicada: {sum(pca_smoteenn.explained_variance_ratio_)*100:.2f}%")

# Crear DataFrame
df_pca_smoteenn = pd.DataFrame({
    'PC1': X_pca_smoteenn[:, 0],
    'PC2': X_pca_smoteenn[:, 1],
    'Mortalidad': pd.Series(y_resampled).map({0: 'Sobrevivió', 1: 'Falleció'})
})

# Visualización PCA con SMOTEENN
fig_pca_smoteenn = px.scatter(
    df_pca_smoteenn, 
    x='PC1', 
    y='PC2', 
    color='Mortalidad',
    color_discrete_map={'Sobrevivió': '#2ecc71', 'Falleció': '#e74c3c'},
    title='Distribución después de SMOTEENN (PCA 2D)',
    labels={'PC1': f'Componente Principal 1 ({pca_smoteenn.explained_variance_ratio_[0]*100:.1f}%)',
            'PC2': f'Componente Principal 2 ({pca_smoteenn.explained_variance_ratio_[1]*100:.1f}%)'},
    opacity=0.6
)

fig_pca_smoteenn.update_layout(
    template='plotly_white',
    width=900,
    height=600,
    legend=dict(title='Estado', yanchor="top", y=0.99, xanchor="right", x=0.99)
)

fig_pca_smoteenn.show()

Aplicando PCA a datos balanceados con SMOTEENN...
Varianza explicada: [0.12183113 0.06267455]
Varianza total explicada: 18.45%


In [15]:
# t-SNE sobre datos con SMOTEENN
print("Calculando t-SNE sobre datos balanceados con SMOTEENN (puede tomar unos segundos)...")
tsne_smoteenn = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne_smoteenn = tsne_smoteenn.fit_transform(X_resampled)

# Crear DataFrame
df_tsne_smoteenn = pd.DataFrame({
    'TSNE1': X_tsne_smoteenn[:, 0],
    'TSNE2': X_tsne_smoteenn[:, 1],
    'Mortalidad': pd.Series(y_resampled).map({0: 'Sobrevivió', 1: 'Falleció'})
})

# Visualización t-SNE con SMOTEENN
fig_tsne_smoteenn = px.scatter(
    df_tsne_smoteenn, 
    x='TSNE1', 
    y='TSNE2', 
    color='Mortalidad',
    color_discrete_map={'Sobrevivió': '#2ecc71', 'Falleció': '#e74c3c'},
    title='Distribución después de SMOTEENN (t-SNE 2D)',
    labels={'TSNE1': 't-SNE Dimensión 1', 'TSNE2': 't-SNE Dimensión 2'},
    opacity=0.6
)

fig_tsne_smoteenn.update_layout(
    template='plotly_white',
    width=900,
    height=600,
    legend=dict(title='Estado', yanchor="top", y=0.99, xanchor="right", x=0.99)
)

fig_tsne_smoteenn.show()

Calculando t-SNE sobre datos balanceados con SMOTEENN (puede tomar unos segundos)...


In [17]:
# Comparación lado a lado: Antes vs Después de SMOTEENN
fig_comparison_smoteenn = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'PCA - Datos Originales', 
        'PCA - Después de SMOTEENN',
        't-SNE - Datos Originales', 
        't-SNE - Después de SMOTEENN'
    ),
    horizontal_spacing=0.08,
    vertical_spacing=0.12
)

# Colores originales
colors_orig = ['#2ecc71' if m == 0 else '#e74c3c' for m in y]
colors_smoteenn = ['#2ecc71' if m == 0 else '#e74c3c' for m in y_resampled]

# PCA Original
fig_comparison_smoteenn.add_trace(
    go.Scatter(x=X_pca[:, 0], y=X_pca[:, 1], mode='markers',
               marker=dict(color=colors_orig, size=4, opacity=0.5), showlegend=False),
    row=1, col=1
)

# PCA SMOTEENN
fig_comparison_smoteenn.add_trace(
    go.Scatter(x=X_pca_smoteenn[:, 0], y=X_pca_smoteenn[:, 1], mode='markers',
               marker=dict(color=colors_smoteenn, size=4, opacity=0.5), showlegend=False),
    row=1, col=2
)

# t-SNE Original
fig_comparison_smoteenn.add_trace(
    go.Scatter(x=X_tsne[:, 0], y=X_tsne[:, 1], mode='markers',
               marker=dict(color=colors_orig, size=4, opacity=0.5), showlegend=False),
    row=2, col=1
)

# t-SNE SMOTEENN
fig_comparison_smoteenn.add_trace(
    go.Scatter(x=X_tsne_smoteenn[:, 0], y=X_tsne_smoteenn[:, 1], mode='markers',
               marker=dict(color=colors_smoteenn, size=4, opacity=0.5), showlegend=False),
    row=2, col=2
)

# Leyenda manual
fig_comparison_smoteenn.add_trace(
    go.Scatter(x=[None], y=[None], mode='markers',
               marker=dict(size=10, color='#2ecc71'), name='Sobrevivió')
)
fig_comparison_smoteenn.add_trace(
    go.Scatter(x=[None], y=[None], mode='markers',
               marker=dict(size=10, color='#e74c3c'), name='Falleció')
)

fig_comparison_smoteenn.update_layout(
    title_text=f'Comparación: Datos Originales vs SMOTEENN<br><sup>Original: {len(y)} muestras | SMOTEENN: {len(y_resampled)} muestras</sup>',
    template='plotly_white',
    width=1100,
    height=900,
    legend=dict(title='Estado', yanchor="top", y=0.99, xanchor="right", x=0.99)
)

# Actualizar etiquetas de ejes
fig_comparison_smoteenn.update_xaxes(title_text='PC1', row=1, col=1)
fig_comparison_smoteenn.update_yaxes(title_text='PC2', row=1, col=1)
fig_comparison_smoteenn.update_xaxes(title_text='PC1', row=1, col=2)
fig_comparison_smoteenn.update_yaxes(title_text='PC2', row=1, col=2)
fig_comparison_smoteenn.update_xaxes(title_text='t-SNE 1', row=2, col=1)
fig_comparison_smoteenn.update_yaxes(title_text='t-SNE 2', row=2, col=1)
fig_comparison_smoteenn.update_xaxes(title_text='t-SNE 1', row=2, col=2)
fig_comparison_smoteenn.update_yaxes(title_text='t-SNE 2', row=2, col=2)

fig_comparison_smoteenn.show()

# Resumen de cambios
print("\n=== Resumen de SMOTEENN ===")
print(f"Muestras originales: {len(y)}")
print(f"  - Sobrevivió: {(y==0).sum()} ({(y==0).sum()/len(y)*100:.1f}%)")
print(f"  - Falleció: {(y==1).sum()} ({(y==1).sum()/len(y)*100:.1f}%)")
print(f"\nMuestras después de SMOTEENN: {len(y_resampled)}")
print(f"  - Sobrevivió: {(y_resampled==0).sum()} ({(y_resampled==0).sum()/len(y_resampled)*100:.1f}%)")
print(f"  - Falleció: {(y_resampled==1).sum()} ({(y_resampled==1).sum()/len(y_resampled)*100:.1f}%)")


=== Resumen de SMOTEENN ===
Muestras originales: 3093
  - Sobrevivió: 2819 (91.1%)
  - Falleció: 274 (8.9%)

Muestras después de SMOTEENN: 3430
  - Sobrevivió: 642 (18.7%)
  - Falleció: 2788 (81.3%)
